In [ ]:
import sys
import os
import numpy as np
#!!!!!!!!!!!!!!!!!!!!!! IMPORTANT  IMPORTANT  IMPORTANT !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# This line is to add the folder of IBridgePy to PYTHONPATH
# Otherwise, this error will show up: ModuleNotFoundError: No module named 'trader_factory'
# For example, Dr. Hui Liu installs IBridgePy at /Users/huil/Documents/Yellowstone
sys.path.append('/Users/huil/Documents/Yellowstone') 

In [ ]:
import pandas as pd
from trader_factory import build_active_IBridgePy_plus, build_trader_for_backtest
from IBridgePy.IbridgepyTools import symbol
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [ ]:
t = build_trader_for_backtest()  # t is the IBridgePy_plus object

In [ ]:
# Retrieve historical data of SPY, daily bar, and go back 8000 days.
hist = t.request_historical_data(symbol('SPY'), '1 day', '8000 D', dataProviderName='YahooFinance')

# In case, yahoo finance is down
# hist = pd.read_csv(os.path.join('Input', 'SPY.csv'), header=0, index_col=0)
# hist.columns = map(str.lower, hist.columns)

In [ ]:
def add_yield(hist):
    hist['close_yesterday'] = hist['close'].shift(1)  # add a new column, call it "close_yesterday"
    hist['close_price_change_from_yesterday_to_today'] = (hist['close'] - hist['close_yesterday']) / hist['close_yesterday']
    hist['close_price_change_from_today_to_tomorrow'] = hist['close_price_change_from_yesterday_to_today'].shift(-1)
    return hist

hist = add_yield(hist)

In [ ]:
def calculate_profit(hist):
    hist['decision'] = hist['close_price_change_from_yesterday_to_today'].apply(lambda x: 1 if x < 0.011 else 0)
    hist['result'] = hist['decision'] * hist['close_price_change_from_today_to_tomorrow']
    hist['portfolio_Strategy'] = np.cumprod(hist['result'] + 1)
    hist['buy_and_hold_SPY'] = np.cumprod(hist['close_price_change_from_today_to_tomorrow'] + 1)
    return hist

hist = calculate_profit(hist)

In [ ]:
hist.tail()

In [ ]:
plt.plot(hist['buy_and_hold_SPY'], color="black", label = "buy_and_hold_SPY")
plt.plot(hist['portfolio_Strategy'], color="red", label = "portfolio_Strategy")
plt.xlabel('Year')
plt.ylabel('Return in Percentage')
plt.legend()

In [ ]:
def machine_learning_calculate_profit(hist, rate):
    hist['decision'] = hist['close_price_change_from_yesterday_to_today'].apply(lambda x: 1 if x < rate else 0)
    hist['result'] = hist['decision'] * hist['close_price_change_from_today_to_tomorrow']
    hist['portfolio_Strategy'] = np.cumprod(hist['result'] + 1)
    hist['buy_and_hold_SPY'] = np.cumprod(hist['close_price_change_from_today_to_tomorrow'] + 1)
    return hist['portfolio_Strategy'][-2]

In [ ]:
ans = {}
for i in range(-20, 20):
    rate = i * 0.001
    ans[rate] = machine_learning_calculate_profit(hist, rate)
print(ans)

In [ ]:
plt.plot(ans.keys(), ans.values())
plt.xlabel('rate')
plt.ylabel('Final Return in Percentage')

In [ ]:
What ideas do you have now?